In [74]:
import os
import pandas as pd
import numpy as np
import sqlalchemy as sqla
import pymysql

In [75]:
user="user"
password="grad5100user"
awsresource="database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"
dbname="topChef"
port = 3306
dburi = f"mysql+pymysql://{user}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)

In [76]:
def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()


def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result

1. There are two types of top Chef programs; the ‘Masters’ programs and the regular series. The database contains information on both. We are not interested in the ‘Masters’. There is also a season called “Canada 6” which is part of the Masters series. Create versions of the chefdetails and judges tables that exclude any records coming from seasons that include the word Masters or Canada.

2. Further trim your chefdetails and judges tables by including only the columns of interest from the table descriptions above.

In [77]:
tables = run_sql("show tables;", engine)
for x in tables:
    print(x)

('challengedescriptions',)
('challengewins',)
('chefdetails',)
('episodeinfo',)
('judges',)
('rewards',)


In [78]:
chef_details_info = run_sql("describe chefdetails;", engine)
for x in chef_details_info[:15]:
    print(x)

('num', 'bigint', 'YES', 'MUL', None, '')
('name', 'text', 'YES', '', None, '')
('chef', 'text', 'YES', '', None, '')
('hometown', 'text', 'YES', '', None, '')
('city', 'text', 'YES', '', None, '')
('state', 'text', 'YES', '', None, '')
('age', 'double', 'YES', '', None, '')
('season', 'text', 'YES', '', None, '')
('seasonNumber', 'bigint', 'YES', '', None, '')
('series', 'text', 'YES', '', None, '')
('placement', 'bigint', 'YES', '', None, '')
('personOfColor', 'text', 'YES', '', None, '')
('occupation', 'text', 'YES', '', None, '')
('gender', 'text', 'YES', '', None, '')


In [89]:
chefdetails = df_select("select * from chefdetails where season NOT LIKE '%Masters%' AND season NOT LIKE 'Canada 6';", engine)
chefdetails

Retrieved 322 records


,num,name,chef,hometown,city,state,age,season,seasonNumber,series,placement,personOfColor,occupation,gender
0,0,Richard Blais,Richard B.,None,None,None,38.0,All Stars: New York,8,US,1,None,Owner,Male
1,1,Mike Isabella,Mike I.,None,None,None,35.0,All Stars: New York,8,US,2,None,Executive Chef/Owner,Male
2,2,Antonia Lofaso,Antonia L.,None,None,None,34.0,All Stars: New York,8,US,3,None,Executive Chef,Female
3,3,Tiffany Derry,Tiffany D.,None,None,None,27.0,All Stars: New York,8,US,4,POC,Executive Chef,Female
4,4,Carla Hall,Carla H.,None,None,None,46.0,All Stars: New York,8,US,5,POC,Chef/Owner,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,386,May Phattanant Thongthong,May Phattanant Thongthong,"Chiang Mai, Thailand",Bangkok,Thailand,NaN,World All Stars,20,US,14,POC,Executive Chef/Co-Partner,Female
318,387,Begona Rodrigo,Begona Rodrigo,"Valencia, Spain",Valencia,Spain,NaN,World All Stars,20,US,12,None,Owner,Female
319,388,Gabriel Rodriguez,Gabriel Rodriguez,"Iztapalapa, Mexico City",Madrid,Spain,NaN,World All Stars,20,US,2,POC,Executive Chef,Male
320,389,Amar Santana,Amar S.,Dominican Republic / Queens,Santa Ana,California,NaN,World All Stars,20,US,6,POC,Chef/Partner,Male


In [79]:
abr_chefdetails = df_select("select season, seasonNumber,name,placement, gender from chefdetails where season NOT LIKE 'Canada 6' and season NOT LIKE '%Masters%';", engine)
abr_chefdetails

Retrieved 322 records


,season,seasonNumber,name,placement,gender
0,All Stars: New York,8,Richard Blais,1,Male
1,All Stars: New York,8,Mike Isabella,2,Male
2,All Stars: New York,8,Antonia Lofaso,3,Female
3,All Stars: New York,8,Tiffany Derry,4,Female
4,All Stars: New York,8,Carla Hall,5,Female
...,...,...,...,...,...
317,World All Stars,20,May Phattanant Thongthong,14,Female
318,World All Stars,20,Begona Rodrigo,12,Female
319,World All Stars,20,Gabriel Rodriguez,2,Male
320,World All Stars,20,Amar Santana,6,Male


In [80]:
judge_info = run_sql("describe judges;", engine)
for x in judge_info[:10]:
    print(x)

('num', 'bigint', 'YES', 'MUL', None, '')
('season', 'text', 'YES', '', None, '')
('seasonNumber', 'bigint', 'YES', '', None, '')
('series', 'text', 'YES', '', None, '')
('episode', 'bigint', 'YES', '', None, '')
('challengeType', 'text', 'YES', '', None, '')
('outcomeType', 'text', 'YES', '', None, '')
('guestJudge', 'text', 'YES', '', None, '')
('competedOnTC', 'text', 'YES', '', None, '')
('otherShows', 'text', 'YES', '', None, '')


In [90]:
judges = df_select("select * from judges where season NOT LIKE '%Masters%' AND season NOT LIKE 'Canada 6';", engine)
judges

Retrieved 633 records


,num,season,seasonNumber,series,episode,challengeType,outcomeType,guestJudge,competedOnTC,otherShows
0,0,All Stars: New York,8,US,1,Quickfire,Team,Tom Colicchio,None,Top Chef Judge
1,1,All Stars: New York,8,US,1,Elimination,Individual,Anthony Bourdain,None,Parts Unknown; No Reservations
2,2,All Stars: New York,8,US,2,Quickfire,Individual,Joe Jonas,None,None
3,3,All Stars: New York,8,US,2,Elimination,Team,Katie Lee,None,Beat Bobby Flay Judge
4,4,All Stars: New York,8,US,3,Quickfire,Team,David Chang,None,None
...,...,...,...,...,...,...,...,...,...,...
628,750,World All Stars,20,US,12,Quickfire,Individual,Sam Bompas,None,None
629,751,World All Stars,20,US,12,Elimination,Individual,Jeremy Chan,None,None
630,752,World All Stars,20,US,13,Quickfire,Individual,Greg Marchand,None,None
631,753,World All Stars,20,US,13,Elimination,Individual,Greg Marchand,None,None


In [81]:
abr_judges = df_select("select season, seasonNumber, episode, challengeType, guestJudge, competedOnTC from judges where season NOT LIKE 'Canada 6' and season NOT LIKE '%Masters%';", engine)
abr_judges

Retrieved 633 records


,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
0,All Stars: New York,8,1,Quickfire,Tom Colicchio,None
1,All Stars: New York,8,1,Elimination,Anthony Bourdain,None
2,All Stars: New York,8,2,Quickfire,Joe Jonas,None
3,All Stars: New York,8,2,Elimination,Katie Lee,None
4,All Stars: New York,8,3,Quickfire,David Chang,None
...,...,...,...,...,...,...
628,World All Stars,20,12,Quickfire,Sam Bompas,None
629,World All Stars,20,12,Elimination,Jeremy Chan,None
630,World All Stars,20,13,Quickfire,Greg Marchand,None
631,World All Stars,20,13,Elimination,Greg Marchand,None


3. Your tables should have information from 20 remaining different seasons, numbered from 1 to 20. Make a table that has two columns, one called season and one called seasonNumber showing the number associated with each season. (For example, San Francisco is season 1.)

In [82]:
seasons_table = df_select("select distinct season, seasonNumber from judges where season NOT LIKE 'Canada 6' and season NOT LIKE '%Masters%' order by seasonNumber;", engine)
seasons_table

Retrieved 21 records


,season,seasonNumber
0,San Francisco,1
1,Los Angeles,2
2,Miami,3
3,Chicago,4
4,New York,5
5,Las Vegas,6
6,D.C.,7
7,All Stars: New York,8
8,Texas,9
9,Seattle,10


4. Answer the following questions using your tables so far.

a. Among all chef contestants, how many are male and how many are female?

Answer: There are 152 women and contestants and 170 male contestants.

b. Among all winners (placement==1) how many are male and how many are female?

Answer: There are 6 female winners and 14 male winners.

c. Among all sets of top 3 finalists (placement=1,2,3) how many are male and how many are female?

Answer: There are 25 female finalists and 36 male finalists.

d. Compare the number of times a female was placed in the top 3 with the number of times a female won. Does this number seem unusual? That is, are women who place in the top 3 less likely than expected to ultimately win the competition? Why or why not?

Answer: Yes, it does seems as though women who place in the top 3 are less likely than expected to ultimately win the competition, as the percentage of women finalists that win is 24% compared to 38.8% of male finalists. 

In [83]:
abr_chefdetails.groupby('gender').count()

,season,seasonNumber,name,placement
gender,,,,
Female,152,152,152,152
Male,170,170,170,170


In [84]:
winners = abr_chefdetails[abr_chefdetails['placement']==1]
winners.groupby('gender').count()

,season,seasonNumber,name,placement
gender,,,,
Female,6,6,6,6
Male,14,14,14,14


In [85]:
finalists = abr_chefdetails[abr_chefdetails['placement'].isin([1,2,3])]
finalists.groupby('gender').count()

,season,seasonNumber,name,placement
gender,,,,
Female,25,25,25,25
Male,36,36,36,36


4. Make a table with one row for each first place winner containing the winner’s name, the season they won, and a column containing ‘Yes’ or ‘No’ depending on whether they served as a guest judge or not.

In [86]:
guest_judge_winners = df_select("""
    select chefdetails.name, chefdetails.season, if(judges.guestJudge is not null, 'Yes', 'No')
    from chefdetails
    left join judges on chefdetails.name = judges.guestJudge and chefdetails.season = judges.season
    where chefdetails.placement = 1
""", engine)

guest_judge_winners

Retrieved 26 records


,name,season,"if(judges.guestJudge is not null, 'Yes', 'No')"
0,Richard Blais,All Stars: New York,No
1,Melissa King,All-Stars L.A.,No
2,Mei Lin,Boston,No
3,Jeremy Ford,California,No
4,Ross Larkin,Canada 6,No
5,Brooke Williamson,Charleston,No
6,Stephanie Izard,Chicago,No
7,Kevin Sbraga,D.C.,No
8,Joseph Flamm,Colorado,No
9,Kah-wai Lo,Houston,No


5. Find all contestants who were not first place winners but did serve as a guest judge at some point. Make sure to only list each name once.

In [87]:
nonwinner_guest_judges = df_select("""
    select chefdetails.name, chefdetails.season, if(judges.guestJudge is not null, 'Yes', 'No')
    from chefdetails
    join judges on chefdetails.name = judges.guestJudge and chefdetails.season = judges.season
    where chefdetails.placement != 1
""", engine)

nonwinner_guest_judges

Retrieved 9 records


,name,season,"if(judges.guestJudge is not null, 'Yes', 'No')"
0,Patricia Yeo,Masters Season 4,Yes
1,Lorena Garcia,Masters Season 4,Yes
2,Kerry Heffernan,Masters Season 4,Yes
3,Lee Anne Wong,San Francisco,Yes
4,Miguel Morales,San Francisco,Yes
5,Stephen Asprinio,San Francisco,Yes
6,Josie Smith-Malave,Seattle,Yes
7,Stefan Richter,Seattle,Yes
8,Sang Yoon,Masters Season 5,Yes
